In [1]:
#Extract the Top 10 Most Reviewed restaurants in Minnesota from Yelp using web scraping

In [2]:
from pprint import pprint
from urllib import request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import pymysql
import datetime

In [3]:
mysql_schema = 'Project_A'
mysql_table = 'yelp_mn_reviews'
file = open('/Users/dhong/Desktop/csv_files/pwd.txt','r')
pwd = file.read()
ifexists_method = 'append'

#Connect to MySQL database 
db = pymysql.connect("localhost", "root", pwd, mysql_schema, charset='utf8')


In [4]:
#Get most reviewed restaurants in Minneapolis
get_review_cnt = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Minneapolis,+MN&start=0&sortby=review_count"

test_url = request.urlopen(get_review_cnt)

read_html = test_url.read()


In [5]:
soup = BeautifulSoup(read_html)

/Users/dhong/anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/dhong/anaconda/envs/py35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
#Save the html code as a string
save_html = soup.prettify()

In [7]:
#Save the string as a html file to my local environment
#I like to view this code in Visual Studio Code since color formatting helps to identify which div tag I should be 
#filtering for in the long line of code.
with open("/Users/dhong/Desktop/yelp.html", "w") as text_file:
    text_file.write(save_html)

In [8]:
#extract all of the code under the div tag "search-results-content". This will reduce the amount of code I need to get
#the top 10 restaurants in Minneapolis with the most reviews.
getsearchresults = soup.html.find_all("div", class_="search-results-content")


In [9]:
#Need to create another set based on the getsearchresults array for the findChildren tag filters.
#findChildren and find_all filters can be used on an element tag
newresults = getsearchresults[0]
type(newresults)

bs4.element.Tag

In [10]:
span_filter = newresults.findChildren('span')
address_filter = newresults.findChildren('address')
a_href_filter = newresults.find_all('a')
category_filter = newresults.find_all("span", {"class": "category-str-list"})

In [11]:
category_list = []

for x in category_filter:
    category_list.append(x.text.replace('\n','').replace(',','|').replace('  ',''))
    
category_list

['American (New)|Breakfast & Brunch|Sandwiches',
 'American (New)|Steakhouses|Beer Gardens',
 'Burgers|Dive Bars',
 'American (New)',
 'Italian|Desserts|Wine Bars',
 'Southern|Caribbean',
 'Vietnamese|Sandwiches|Noodles',
 'Pizza',
 'Pubs|Irish',
 'Italian|Seafood|American (Traditional)']

In [12]:
span_filter2 = []

for x in span_filter:
    if "category-str-list" not in str(x):
        span_filter2.append(x)
    

In [13]:
getaddress = [] 
for x in address_filter:
    getaddress.append(x.text.lstrip().rstrip())

address_list = []
for x in getaddress:
    address_list.append(str(x).replace('Minneapolis, MN',' Minneapolis, MN'))
    
address_list

['80 S 9th St Minneapolis, MN 55402',
 '1121 Hennepin Ave Minneapolis, MN 55403',
 '3500 Cedar Ave S Minneapolis, MN 55407',
 '112 N 3rd St Minneapolis, MN 55401',
 '800 Washington Ave N Minneapolis, MN 55401',
 '600 E Hennepin Ave Minneapolis, MN 55414',
 '2719 Nicollet Ave S Minneapolis, MN 55408',
 '5557 Xerxes Ave S Minneapolis, MN 55410',
 '931 Nicollet Mall Minneapolis, MN 55402',
 '831 Nicollet Mall Minneapolis, MN 55402']

In [14]:
#Use zip to merge the lists of address and Food category
merge_list = []

for x in zip(address_list, category_list):
    merge_list.append(x)

merge_list

[('80 S 9th St Minneapolis, MN 55402',
  'American (New)|Breakfast & Brunch|Sandwiches'),
 ('1121 Hennepin Ave Minneapolis, MN 55403',
  'American (New)|Steakhouses|Beer Gardens'),
 ('3500 Cedar Ave S Minneapolis, MN 55407', 'Burgers|Dive Bars'),
 ('112 N 3rd St Minneapolis, MN 55401', 'American (New)'),
 ('800 Washington Ave N Minneapolis, MN 55401', 'Italian|Desserts|Wine Bars'),
 ('600 E Hennepin Ave Minneapolis, MN 55414', 'Southern|Caribbean'),
 ('2719 Nicollet Ave S Minneapolis, MN 55408',
  'Vietnamese|Sandwiches|Noodles'),
 ('5557 Xerxes Ave S Minneapolis, MN 55410', 'Pizza'),
 ('931 Nicollet Mall Minneapolis, MN 55402', 'Pubs|Irish'),
 ('831 Nicollet Mall Minneapolis, MN 55402',
  'Italian|Seafood|American (Traditional)')]

In [15]:
#Cleanse the data from the get_top_ten list by removing unwanted characters
get_top_ten = [] 
for x in span_filter2:
    get_top_ten.append(x.text.lstrip().rstrip())

get_top_ten = str(get_top_ten).replace('\\n','').replace('  ','').replace('[','').replace(']','').replace("'","") \
                      .replace('Phone number,','').replace('reviews','').split(',')
get_top_ten

['1. Hell’s Kitchen',
 ' Hell’s Kitchen',
 ' 1505 ',
 ' $$',
 ' $$',
 ' Downtown Minneapolis',
 '  (612) 332-4700',
 ' 2. Butcher & The Boar',
 ' Butcher & The Boar',
 ' 1055 ',
 ' $$$',
 ' $$$',
 ' Downtown Minneapolis',
 '  (612) 238-8888',
 ' 3. Matt’s Bar',
 ' Matt’s Bar',
 ' 972 ',
 ' $',
 ' $',
 ' Powderhorn',
 '  (612) 722-7072',
 ' 4. 112 Eatery',
 ' 112 Eatery',
 ' 903 ',
 ' $$$',
 ' $$$',
 ' Downtown Minneapolis',
 '  (612) 343-7696',
 ' 5. Bar La Grassa',
 ' Bar La Grassa',
 ' 884 ',
 ' $$$',
 ' $$$',
 ' North Loop',
 '  (612) 333-3837',
 ' 6. Brasa Rotisserie',
 ' Brasa Rotisserie',
 ' 778 ',
 ' $$',
 ' $$',
 ' Northeast',
 '  (612) 379-3030',
 ' 7. Quang Restaurant',
 ' Quang Restaurant',
 ' 684 ',
 ' $',
 ' $',
 ' Whittier',
 '  (612) 870-4739',
 ' 8. Pizzeria Lola',
 ' Pizzeria Lola',
 ' 679 ',
 ' $$',
 ' $$',
 ' Southwest',
 '  (612) 424-8338',
 ' 9. The Local',
 ' The Local',
 ' 557 ',
 ' $$',
 ' $$',
 ' Downtown Minneapolis',
 '  (612) 904-1000',
 ' 10. Zelo',
 ' Zelo

In [16]:
#Create empty list called data and append 7 values for each array
data = []

start = 0
incr = start + 7
end = len(get_top_ten) - 2

for x in get_top_ten:
    while start < end:
        data.append(get_top_ten[start:incr])
        
        start = start + 7
        incr = start + 7

data

[['1. Hell’s Kitchen',
  ' Hell’s Kitchen',
  ' 1505 ',
  ' $$',
  ' $$',
  ' Downtown Minneapolis',
  '  (612) 332-4700'],
 [' 2. Butcher & The Boar',
  ' Butcher & The Boar',
  ' 1055 ',
  ' $$$',
  ' $$$',
  ' Downtown Minneapolis',
  '  (612) 238-8888'],
 [' 3. Matt’s Bar',
  ' Matt’s Bar',
  ' 972 ',
  ' $',
  ' $',
  ' Powderhorn',
  '  (612) 722-7072'],
 [' 4. 112 Eatery',
  ' 112 Eatery',
  ' 903 ',
  ' $$$',
  ' $$$',
  ' Downtown Minneapolis',
  '  (612) 343-7696'],
 [' 5. Bar La Grassa',
  ' Bar La Grassa',
  ' 884 ',
  ' $$$',
  ' $$$',
  ' North Loop',
  '  (612) 333-3837'],
 [' 6. Brasa Rotisserie',
  ' Brasa Rotisserie',
  ' 778 ',
  ' $$',
  ' $$',
  ' Northeast',
  '  (612) 379-3030'],
 [' 7. Quang Restaurant',
  ' Quang Restaurant',
  ' 684 ',
  ' $',
  ' $',
  ' Whittier',
  '  (612) 870-4739'],
 [' 8. Pizzeria Lola',
  ' Pizzeria Lola',
  ' 679 ',
  ' $$',
  ' $$',
  ' Southwest',
  '  (612) 424-8338'],
 [' 9. The Local',
  ' The Local',
  ' 557 ',
  ' $$',
  ' $$',

In [17]:
#Remove last array from the data list
data.pop()

[' 1', ' Next', ' ']

In [18]:
#Get current date
now = datetime.datetime.now()

In [19]:
#Create pandas dataframe from the lists
df = pd.DataFrame({
        'RESTAURANT':[x[1] for x in data],
        'REVIEWS':[x[2] for x in data],
        'PRICE_RANGE':[x[3] for x in data],
        'AREA':[x[5] for x in data],
        'PHONE':[x[6] for x in data],
        'ADDRESS':[x[0] for x in merge_list],
        'CATEGORY':[x[1] for x in merge_list],
        'DATE': now.strftime("%Y-%m-%d")
    })



df

,ADDRESS,AREA,CATEGORY,DATE,PHONE,PRICE_RANGE,RESTAURANT,REVIEWS
0,"80 S 9th St Minneapolis, MN 55402",Downtown Minneapolis,American (New)|Breakfast & Brunch|Sandwiches,2017-02-28,(612) 332-4700,$$,Hell’s Kitchen,1505
1,"1121 Hennepin Ave Minneapolis, MN 55403",Downtown Minneapolis,American (New)|Steakhouses|Beer Gardens,2017-02-28,(612) 238-8888,$$$,Butcher & The Boar,1055
2,"3500 Cedar Ave S Minneapolis, MN 55407",Powderhorn,Burgers|Dive Bars,2017-02-28,(612) 722-7072,$,Matt’s Bar,972
3,"112 N 3rd St Minneapolis, MN 55401",Downtown Minneapolis,American (New),2017-02-28,(612) 343-7696,$$$,112 Eatery,903
4,"800 Washington Ave N Minneapolis, MN 55401",North Loop,Italian|Desserts|Wine Bars,2017-02-28,(612) 333-3837,$$$,Bar La Grassa,884
5,"600 E Hennepin Ave Minneapolis, MN 55414",Northeast,Southern|Caribbean,2017-02-28,(612) 379-3030,$$,Brasa Rotisserie,778
6,"2719 Nicollet Ave S Minneapolis, MN 55408",Whittier,Vietnamese|Sandwiches|Noodles,2017-02-28,(612) 870-4739,$,Quang Restaurant,684
7,"5557 Xerxes Ave S Minneapolis, MN 55410",Southwest,Pizza,2017-02-28,(612) 424-8338,$$,Pizzeria Lola,679
8,"931 Nicollet Mall Minneapolis, MN 55402",Downtown Minneapolis,Pubs|Irish,2017-02-28,(612) 904-1000,$$,The Local,557
9,"831 Nicollet Mall Minneapolis, MN 55402",Downtown Minneapolis,Italian|Seafood|American (Traditional),2017-02-28,(612) 333-7000,$$,Zelo,540


In [20]:
#Load pandas dataframe to MySQL table
df.to_sql(con=db, name=mysql_table, index=False, if_exists=ifexists_method, flavor='mysql')

/Users/dhong/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


In [21]:
#Query from MySQL to see the results
cur = db.cursor()
cur.execute("select * from project_a.yelp_mn_reviews")

for row in cur:
    print(row)

('80 S 9th St Minneapolis, MN 55402', ' Downtown Minneapolis', 'American (New)|Breakfast & Brunch|Sandwiches', '2017-02-28', '  (612) 332-4700', ' $$', ' Hell’s Kitchen', ' 1505 ')
('1121 Hennepin Ave Minneapolis, MN 55403', ' Downtown Minneapolis', 'American (New)|Steakhouses|Beer Gardens', '2017-02-28', '  (612) 238-8888', ' $$$', ' Butcher & The Boar', ' 1055 ')
('3500 Cedar Ave S Minneapolis, MN 55407', ' Powderhorn', 'Burgers|Dive Bars', '2017-02-28', '  (612) 722-7072', ' $', ' Matt’s Bar', ' 972 ')
('112 N 3rd St Minneapolis, MN 55401', ' Downtown Minneapolis', 'American (New)', '2017-02-28', '  (612) 343-7696', ' $$$', ' 112 Eatery', ' 903 ')
('800 Washington Ave N Minneapolis, MN 55401', ' North Loop', 'Italian|Desserts|Wine Bars', '2017-02-28', '  (612) 333-3837', ' $$$', ' Bar La Grassa', ' 884 ')
('600 E Hennepin Ave Minneapolis, MN 55414', ' Northeast', 'Southern|Caribbean', '2017-02-28', '  (612) 379-3030', ' $$', ' Brasa Rotisserie', ' 778 ')
('2719 Nicollet Ave S Minnea